In [1]:
from googlesearch import search

In [2]:
query = "brooklyn union"
results_list = []
for i in search(query,        # The query you want to run
                tld = 'com',  # The top level domain
                lang = 'en',  # The language
                num = 10,     # Number of results per page
                start = 0,    # First result to retrieve
                stop = 10,  # Last result to retrieve
              
               ):
    results_list.append(i)
    print(i)

https://www.nationalgridus.com/
https://www1.nationalgridus.com/signin
https://www.nationalgridus.com/contact-us
https://www.nationalgridus.com/NY-Home/
https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay
https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html
https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html#credit-reports
https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html#related-companies
https://en.wikipedia.org/wiki/KeySpan
https://www.bloomberg.com/profile/company/80132MF:US


In [3]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlparse, urljoin
import urllib.robotparser
import urllib.request
import queue

In [13]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url("http://google.com/robots.txt")
rp.read()
print(rp.can_fetch("*","http://google.com/search/"))
print(rp.can_fetch("*","http://google.com/"))

False
False


In [16]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url("http://nyu.edu/robots.txt")
rp.read()
print(rp.can_fetch("*","http://nyu.edu/search/"))
print(rp.can_fetch("*","http://nyu.edu/"))
# print(rp.can_fetch("*","http://google.com/search/"))
# print(rp.can_fetch("*","http://google.com/"))

False
False


In [13]:

html_page = requests.get("http://google.com/intl/en/ads/")

soup = BeautifulSoup(html_page.text, "html.parser")

urls = []
for link in soup.find_all('a'):
    print(link.get('href'))
print(urls)


None
../home/
#jump-content
../home/how-it-works/
../home/pricing/
../home/faq/
../home/resources/advanced/
../home/contact-us/
tel:18442452553
None
None
https://ads.google.com/nav/login?subid=US-en-et-g-aw-a-home-awhp_xin1_signin!o2
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_top!o2
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_top!o2
../home/
#jump-content
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_hero!o2
None
tel:18442452553
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_mid!o2
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_mid!o2
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_mobmid!o2
../home/pricing/
../home/how-it-works/
https://ads.google.com/signup?subid=US-en-et-g-aw-a-home-awhp_xin1_end!o2
tel:18442452553
None
None
https://crq8j.app.goo.gl/Qjxq
https://policies.google.com/terms
https://policies.google.com/privacy
https://blog.google/products/ads/
h

In [ ]:
# download robots.txt
# multi threading
# download html for each page


In [4]:
def crawl(url):
    try:
        page=urllib.request.urlopen(url)
    except urllib.error.HTTPError as error:
        return url+" "+str(error.code)
    size=len(page.read())/1024
    return url+" Size: "+"{:.2f}".format(size)+"kb"
print(crawl("http://mit.edu"))
print(crawl("https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html"))

http://mit.edu Size: 46.43kb
https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html 403


In [7]:
def get_links(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    links = []
    base = soup.find_all('base')
    if base:
        url=base[0].get('href')
    for link in soup.find_all('a'):
        #filter the links
        #non-url
        #modify relative url to common url
        href=link.get('href')
        parse_result=urlparse(href)
        if parse_result.scheme=="":
            links.append(urljoin(url,href))
        else:
            links.append(href)
    return links
# get_links('http://nyu.edu')
get_links("https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html")

In [6]:
def bfs_crawler():
    seen=set()
    result=[]
    crawled_site={}
    q=queue.Queue()
    for link in results_list:
        parse_result=urlparse(link)
        parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
        netloc=parse_result.netloc
        if parsed_link not in seen:
            if netloc not in crawled_site or crawled_site[netloc]<10:
                q.put(link)
                seen.add(link)
                if netloc not in crawled_site:
                    crawled_site[netloc]=1
                else:
                    crawled_site[netloc]+=1
    while len(result)<20:
        cur=q.get()
        print("crawling",cur)
        print(crawl(cur))
        
        for link in get_links(cur):
            if link:
                parse_result=urlparse(link)
                parsed_link=urljoin(parse_result.scheme+"://"+parse_result.netloc,parse_result.path)
                netloc=parse_result.netloc
                if parsed_link not in seen:
                    if netloc not in crawled_site or crawled_site[netloc]<10:
                        q.put(link)
                        seen.add(link)
                        if netloc not in crawled_site:
                            crawled_site[netloc]=1
                        else:
                            crawled_site[netloc]+=1
bfs_crawler()

crawling https://www.nationalgridus.com/
https://www.nationalgridus.com/ Size: 118.49kb
crawling https://www1.nationalgridus.com/signin
https://www1.nationalgridus.com/signin Size: 167.03kb
crawling https://www.nationalgridus.com/contact-us
https://www.nationalgridus.com/contact-us Size: 119.30kb
crawling https://www.nationalgridus.com/NY-Home/
https://www.nationalgridus.com/NY-Home/ Size: 125.61kb
crawling https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay
https://www.nationalgridus.com/regionalized/Billing-Payments/Ways-to-Pay Size: 118.49kb
crawling https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html
https://www.dnb.com/business-directory/company-profiles.the_brooklyn_union_gas_company.89887e0b5ad7b8157fc9e5f233efd2a8.html 403


ConnectionError: ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))

In [ ]:
#priority queue
#class ScoredUrl

novelty_res={}
candidate_urls={}
url_pos={}
class ScoredUrl:
    def __init__(self, url):
        self.url=url
        self.netloc=urlparse(url).netloc
        self.importance=0
        if self.netloc not in novelty_res:
            novelty_res[self.netloc]=10

    def __eq__(self,other):
        return novelty_res[self.netloc]+self.importance==novelty_res[other.netloc]+other.importance

    def __lt__(self,other):
        return novelty_res[self.netloc]+self.importance<novelty_res[other.netloc]+other.importance

    def __gt__(self,other):
        return self>other

    def add_importance(self):
        self.importance+=1

    def crawl(self):
        return get_links(self.url)



class UrlQueue:
    def __init__(self):
        self.res=[]
    
    def add(self,new_url):
        heapq.heappush(self.res,new_url)
    
    def update(self,new_url):
        while
    #update